# Salary Binary Classification
This problem uses the census income data-set from the Cal Univ of Irvine Machine Learning Repo. Data-set is extracted from the U.S. Census Bureau database. Using this dataset we will classify if a person has a salary if a person has a salary of greater than $50k. It is strictly a binary classification.

In [ ]:
# kaggle/python Docker image: https://github.com/kaggle/docker-python
import numpy as np 
import pandas as pd 

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
cols = [
    'Age', 'Workclass', 'fnlwgt', 'Education', 'Education-num', 
    'Marital-status', 'Occupation', 'Relationship', 'Race', 'Sex', 
    'CG', 'CL', 'HPW', 'Country', 'Salary'
]
unwanted_cols = [
    'fnlwgt', 'Education', 'Relationship', 'CG', 'CL', 'Country'
]

Read data. Drop unwanted columns. Replace '?' with 'nan'. Drop na.

In [ ]:
salary_data = pd.read_csv('../input/adult-census-income/adult.csv', names=cols, header=0).drop(unwanted_cols, axis=1).replace('?', np.nan).dropna()
salary_features = salary_data.drop('Salary', axis=1)

Convert data to numeric form (0 or 1)

In [ ]:
label_encoder = preprocessing.LabelEncoder()

# salary_features[['Sex']] = salary_features[['Sex']].apply(label_encoder.fit_transform)
salary_target = salary_data[['Salary']].apply(label_encoder.fit_transform)

Apply one-hot-encoding

In [ ]:
ohe_cols = ['Workclass', 'Marital-status', 'Occupation', 'Race', 'Sex']
salary_features = pd.get_dummies(salary_features, columns=ohe_cols)

Normalize numerical data

In [ ]:
scalable_cols = ['Age', 'Education-num', 'HPW']
salary_features[scalable_cols] = preprocessing.scale(salary_features[scalable_cols])

Split into train and test sets

In [ ]:
X_train, x_test, Y_train, y_test = train_test_split(
    salary_features,               
    salary_target,
    test_size=0.20,
    random_state=0
)

Training and test data into PyTorch Tensors

In [ ]:
Xtrain_ = torch.from_numpy(X_train.values).float()
Xtest_ = torch.from_numpy(x_test.values).float()
Xtrain_.shape

Reshape our data to match the y-label format that our loss function requires

In [ ]:
Ytrain_ = torch.from_numpy(Y_train.values).view(1, -1)[0].type(torch.LongTensor) # reshape to 1-D tensor with all data in 1 row
Ytest_ = torch.from_numpy(y_test.values).view(1, -1)[0].type(torch.LongTensor)
Ytrain_.type()

Create NN

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(38, 32)       
        self.fc2 = nn.Linear(32, 32)   
        self.fc3 = nn.Linear(32, 2)
        
    def forward(self, x): 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return F.log_softmax(x, dim=-1)

In [ ]:
model = Net()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss() 

In [ ]:
epoch_data = []
epochs = 1000

for epoch in range(1, epochs + 1):
    optimizer.zero_grad()
    
    Ypred = model(Xtrain_)
    
    loss = loss_fn(Ypred, Ytrain_)
    loss.backward() 
    
    optimizer.step()
    
    Ypred_test = model(Xtest_)
    loss_test = loss_fn(Ypred_test, Ytest_)
    
    _,pred = Ypred_test.data.max(1)
    
    accuracy = pred.eq(Ytest_.data).sum().item() / y_test.values.size
    epoch_data.append([epoch, loss.data.item(), loss_test.data.item(), accuracy])
    
    if epoch % 100 == 0:
        print(f'epoch - {epoch} ({epoch/100 * 10}%) train loss - {loss.data.item():.4f} test loss - {loss_test.data.item():.4f} accuracy - {accuracy:.4f}')